## **Presented By:** 
### - Azzouzi Ousssama Abderraouf
### - Mohamed Saadi

# ===============================================================

# 1- Imports

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping


# 2- loading datasets

In [79]:
data_path = '011/text_data/'

ada_joy = data_path + 'ada_joy.csv'
assistance_info = data_path + 'assistance_info.csv'
gaze_positions = data_path + 'gaze_positions.csv'
input_info = data_path + 'input_info.csv'
joint_positions = data_path + 'joint_positions.csv'
myo_emg = data_path + 'myo_emg.csv'
myo_imu = data_path + 'myo_imu.csv'
myo_ori = data_path + 'myo_ori.csv'
pupil_cal_eye0 = data_path + 'pupil_cal_eye0.csv'
pupil_cal_eye1 = data_path + 'pupil_cal_eye1.csv'
pupil_eye0 = data_path + 'pupil_eye0.csv'
pupil_eye1 = data_path + 'pupil_eye1.csv'
robot_position = data_path + 'robot_position.csv'
world_cal_positions = data_path + 'world_cal_positions.csv'

ada_joy_df = pd.read_csv(ada_joy)
assistance_info_df = pd.read_csv(assistance_info)
gaze_positions_df = pd.read_csv(gaze_positions)
input_info_df = pd.read_csv(input_info)
joint_positions_df = pd.read_csv(joint_positions)
myo_emg_df = pd.read_csv(myo_emg)
myo_imu_df = pd.read_csv(myo_imu)
myo_ori_df = pd.read_csv(myo_ori)
pupil_cal_eye0_df = pd.read_csv(pupil_cal_eye0)
pupil_cal_eye1_df = pd.read_csv(pupil_cal_eye1)
pupil_eye0_df = pd.read_csv(pupil_eye0)
pupil_eye1_df = pd.read_csv(pupil_eye1)
robot_position_df = pd.read_csv(robot_position)
world_cal_positions_df = pd.read_csv(world_cal_positions)

ada_joy_df.head(), assistance_info_df.head(), gaze_positions_df.head(), input_info_df.head(), joint_positions_df.head(), myo_emg_df.head(), myo_imu_df.head(), myo_ori_df.head(), pupil_cal_eye0_df.head(), pupil_cal_eye1_df.head(), pupil_eye0_df.head(), pupil_eye1_df.head(), robot_position_df.head(), world_cal_positions_df.head()



(   timestamp  world_index  world_index_corrected    axes_x  axes_y  axes_z  \
 0   5.243769          147                    156 -0.000000    -0.0    -0.0   
 1   5.251749          147                    156 -0.000000    -0.0    -0.0   
 2   5.259806          148                    157 -0.000000    -0.0    -0.0   
 3   5.267750          148                    157  0.004807    -0.0    -0.0   
 4   5.275674          148                    157  0.026684    -0.0    -0.0   
 
    buttons_0  buttons_1  
 0          0          0  
 1          0          0  
 2          0          0  
 3          0          0  
 4          0          0  ,
    timestamp  exec_twist_x  exec_twist_y  exec_twist_z  exec_twist_r  \
 0   1.114619           0.0           0.0           0.0          -0.0   
 1   1.140355           0.0           0.0           0.0          -0.0   
 2   1.165112           0.0           0.0           0.0          -0.0   
 3   1.190987           0.0           0.0           0.0          -0.0

# 3- Data Preprocessing

In [80]:
# # print the column names of the dataframes
# print('ada_joy_df columns: ', ada_joy_df.columns)
# print('assistance_info_df columns: ', assistance_info_df.columns)
# print('gaze_positions_df columns: ', gaze_positions_df.columns)
# print('input_info_df columns: ', input_info_df.columns)
# print('joint_positions_df columns: ', joint_positions_df.columns)
# print('myo_emg_df columns: ', myo_emg_df.columns)
# print('myo_imu_df columns: ', myo_imu_df.columns)
# print('myo_ori_df columns: ', myo_ori_df.columns)
# print('pupil_cal_eye0_df columns: ', pupil_cal_eye0_df.columns)
# print('pupil_cal_eye1_df columns: ', pupil_cal_eye1_df.columns)
# print('pupil_eye0_df columns: ', pupil_eye0_df.columns)
# print('pupil_eye1_df columns: ', pupil_eye1_df.columns)
# print('robot_position_df columns: ', robot_position_df.columns)
# print('world_cal_positions_df columns: ', world_cal_positions_df.columns)


ada_joy_df columns:  Index(['timestamp', 'world_index', 'world_index_corrected', 'axes_x', 'axes_y',
       'axes_z', 'buttons_0', 'buttons_1'],
      dtype='object')
assistance_info_df columns:  Index(['timestamp', 'exec_twist_x', 'exec_twist_y', 'exec_twist_z',
       'exec_twist_r', 'exec_twist_p', 'exec_twist_yaw', 'p_goal_0_raw',
       'p_goal_1_raw', 'p_goal_2_raw', 'p_goal_0', 'p_goal_1', 'p_goal_2'],
      dtype='object')
gaze_positions_df columns:  Index(['timestamp', 'world_index', 'world_index_corrected', 'confidence',
       'norm_pos_x', 'norm_pos_y', 'base_data', 'gaze_point_3d_x',
       'gaze_point_3d_y', 'gaze_point_3d_z', 'eye_center0_3d_x',
       'eye_center0_3d_y', 'eye_center0_3d_z', 'gaze_normal0_x',
       'gaze_normal0_y', 'gaze_normal0_z', 'eye_center1_3d_x',
       'eye_center1_3d_y', 'eye_center1_3d_z', 'gaze_normal1_x',
       'gaze_normal1_y', 'gaze_normal1_z'],
      dtype='object')
input_info_df columns:  Index(['timestamp', 'robot_mode', 'twist_x', 'tw

In [70]:
# Combine datasets based on timestamp (outer join to keep all timestamps)
combined_df = pd.merge(ada_joy_df, gaze_positions_df, on='timestamp', how='outer')
combined_df = pd.merge(combined_df, myo_emg_df, on='timestamp', how='outer')

# Handling missing values (example: fill with mean)
for col in combined_df.columns:
    try:
        combined_df[col] = combined_df[col].astype(float)
    except ValueError:
        print(f"Could not convert column {col} to float")

combined_df = combined_df.fillna(combined_df.select_dtypes(include=[np.number]).mean())
combined_df = combined_df.fillna(0)
combined_df = combined_df.drop(columns=['base_data'])

#  calculate the mean of each numeric column and convert to double
# mean = combined_df.select_dtypes(include=[np.number]).mean()
mean = combined_df.mean()

mean = mean.astype(float)

# print(mean)  # use dtype instead of type()

# Check for missing values
# print(combined_df.isnull().sum())

# combined_df.head(), 
print(combined_df.columns)

Could not convert column base_data to float
Index(['timestamp', 'world_index_x', 'world_index_corrected_x', 'axes_x',
       'axes_y', 'axes_z', 'buttons_0', 'buttons_1', 'world_index_y',
       'world_index_corrected_y', 'confidence', 'norm_pos_x', 'norm_pos_y',
       'gaze_point_3d_x', 'gaze_point_3d_y', 'gaze_point_3d_z',
       'eye_center0_3d_x', 'eye_center0_3d_y', 'eye_center0_3d_z',
       'gaze_normal0_x', 'gaze_normal0_y', 'gaze_normal0_z',
       'eye_center1_3d_x', 'eye_center1_3d_y', 'eye_center1_3d_z',
       'gaze_normal1_x', 'gaze_normal1_y', 'gaze_normal1_z', 'world_index',
       'world_index_corrected', 'emg0', 'emg1', 'emg2', 'emg3', 'emg4', 'emg5',
       'emg6', 'emg7', 'moving'],
      dtype='object')


In [71]:
moving_df = combined_df.loc[combined_df['moving'] != 0]
moving_df.head()

,timestamp,world_index_x,world_index_corrected_x,axes_x,axes_y,axes_z,buttons_0,buttons_1,world_index_y,world_index_corrected_y,...,world_index_corrected,emg0,emg1,emg2,emg3,emg4,emg5,emg6,emg7,moving


# 4- Feature Selection

In [72]:
# Assuming 'moving' is your target variable, you can adjust it accordingly
target_variable = ('gaze_point_3d_x', 'gaze_point_3d_y', 'gaze_point_3d_z')

# Identify redundant features and drop them (you may customize this based on your data)
redundant_features = ['world_index_x', 'world_index_corrected_x', 'world_index_y', 'world_index_corrected_y']
redundant_features = [col for col in redundant_features if col in combined_df.columns]  # Check if columns exist
combined_df = combined_df.drop(columns=redundant_features)

# Select features and target variable
features = combined_df.drop(columns=list(target_variable) + ['timestamp'])
target = combined_df[list(target_variable)]

# Check for highly correlated features and drop if necessary
correlation_matrix = features.corr()
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.95)]
features = features.drop(columns=to_drop)

# Confirm the selected features and target variable
print("Selected Features:", features.columns)
print("Target Variable:", target_variable)

Selected Features: Index(['axes_x', 'axes_y', 'axes_z', 'buttons_0', 'buttons_1', 'confidence',
       'norm_pos_x', 'norm_pos_y', 'eye_center0_3d_x', 'eye_center0_3d_z',
       'gaze_normal0_y', 'gaze_normal0_z', 'eye_center1_3d_x',
       'eye_center1_3d_y', 'gaze_normal1_x', 'gaze_normal1_y',
       'gaze_normal1_z', 'world_index', 'emg0', 'emg1', 'emg2', 'emg3', 'emg4',
       'emg5', 'emg6', 'emg7', 'moving'],
      dtype='object')
Target Variable: ('gaze_point_3d_x', 'gaze_point_3d_y', 'gaze_point_3d_z')


# 5- Data Splitting

In [75]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Confirm the shapes of the sets
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)
print("Training set shape:", y_train.shape)
print("Testing set shape:", y_test.shape)


Training set shape: (19312, 27)
Testing set shape: (4829, 27)
Training set shape: (19312, 3)
Testing set shape: (4829, 3)


# 4- Model Selection and Training

In [76]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the model
model = RandomForestRegressor(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the training set
train_predictions = model.predict(X_train)

# Predict on the testing set
test_predictions = model.predict(X_test)

# Evaluate the model
train_rmse = mean_squared_error(y_train, train_predictions, squared=False)
test_rmse = mean_squared_error(y_test, test_predictions, squared=False)

print("Training RMSE:", train_rmse)
print("Testing RMSE:", test_rmse)

Training RMSE: 1.8493735361686088
Testing RMSE: 1.857122082671694
